In [18]:
import numpy as np
import pandas as pd

# Lab 9 - Multi-layer Perceptron Forward Pass & Backpropagation

## Part I
For this exercise you will implement a simple 2-layer perceptron with the forward pass and the backpropagation to learn the weights

For the first part you'll build and train a 2-layer neural network that predicts the prices of houses, using the usual Boston housing dataset.

In [19]:
boston = pd.read_csv('data/BostonHousing.txt')
boston.head(5)

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


As usual, consider the MEDV as your target variable. 
* Split the data into training, validation and testing (70,15,15)%
* Experiment with different number of neurons per layer for your network, using the validation set

In [20]:
# your code goes here
from sklearn.model_selection import train_test_split

X = boston.drop('medv', axis=1)
y = boston['medv']

features = X.shape[1]
predictions = 1

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.30, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.50, random_state=42)

In [21]:
class MyStandardScaler:
    def fit(self, X):
        self.mean_ = np.mean(X, axis=0)
        self.scale_ = np.std(X, axis=0)
    
    def transform(self, X):
        return (X - self.mean_) / self.scale_
    
    def fit_transform(self, X):
        self.fit(X)
        return self.transform(X)
    
X_train_arr = X_train.values
y_train_arr = y_train.values

scaler = MyStandardScaler()
X_train_normalized = np.array(scaler.fit_transform(X_train_arr))

In [22]:
# activation function and its derivative
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_prime(x):
    return sigmoid(x) * (1 - sigmoid(x))

def identity(x):
    return x

def identity_prime(x):
    return 1

# loss function and its derivative
def weight_decay(weights, alpha):
    return alpha * np.dot(weights.T, weights)/2

def weight_decay_prime(weights, alpha):
    return alpha * weights

def mse(y_true, y_pred):
    return np.mean(np.power(y_true-y_pred, 2))

def mse_prime(y_true, y_pred):
    return (y_pred - y_true)/y_true.size # can be used to mini-batch

def mse_wd(y_true, y_pred, alpha, weights):
    return mse(y_true, y_pred) + weight_decay(weights, alpha)

def mse_wd_prime(y_true, y_pred, alpha, weights):
    return mse_prime(y_true, y_pred) + weight_decay_prime(weights, alpha)

In [23]:
class FCLayer():
    def __init__(self, input_size, output_size, activation, activation_prime):
        self.input = None
        self.output = None
        self.output_without_activation = None
        self.weights = np.random.rand(input_size, output_size) - 0.5
        self.bias = np.random.rand(1, output_size) - 0.5
        self.activation = activation
        self.activation_prime = activation_prime

    # forward propagation through the layer
    def forward_propagation(self, input_data):
        self.input = input_data.reshape(1, -1)
        self.output_without_activation = np.dot(self.input, self.weights) + self.bias
        self.output = self.activation(self.output_without_activation)
        return self.output

    # computes dE/dW (weights), dE/dB (bias) for a given delta_k, from the subsequent layer. Returns delta_j, from this layer.
    def backward_propagation(self, output_error, learning_rate, alpha):
        delta_k_activated = self.activation_prime(self.output_without_activation) * output_error
        delta_j = np.dot(delta_k_activated, self.weights.T)
        gradiente = np.dot(self.input.T, delta_k_activated)
        
        # Regularization term
        self.weight_decay_term = alpha * self.weights

        # update parameters
        self.weights -= learning_rate * gradiente
        self.bias -= learning_rate * delta_k_activated # 1 * delta_k_activated, because input is 1 for bias
        return delta_j

In [46]:
class Network:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_prime = None
        self.alpha = None
        self.errors_during_train = []

    # add layer to network
    def add(self, layer):
        self.layers.append(layer)

    # set loss to use
    def use(self, loss, loss_prime, alpha = 0.1):
        self.loss = loss
        self.loss_prime = loss_prime
        self.alpha = alpha

    # predict output for given input
    def predict(self, input_data):
        samples = len(input_data)
        result = []

        # run network over all samples
        for i in range(samples):
            # forward propagation
            output = input_data[i]
            for layer in self.layers:
                output = layer.forward_propagation(output)
            result.append(output)

        return result

    # train the network
    def fit(self, x_train, y_train, epochs, learning_rate, print_erro = False):
        # sample dimension first
        samples = len(x_train)

        # training loop
        for i in range(epochs):
            loss_train = 0
            for j in range(samples):
                # forward propagation
                output = x_train[j]
                for layer in self.layers:
                    output = layer.forward_propagation(output)

                # compute loss (for display purpose only)
                loss_train = self.loss(y_train[j], output)

                # backward propagation
                error = self.loss_prime(y_train[j], output)
                for layer in reversed(self.layers):
                    error = layer.backward_propagation(error, learning_rate, self.alpha)

            # calculate average error on all samples
            loss_train /= samples
            self.errors_during_train.append(loss_train)
            if print_erro: print('epoch %d/%d   error=%f' % (i+1, epochs, loss_train))

In [47]:
# network
net = Network()
net.add(FCLayer(features, 15, sigmoid, sigmoid_prime))
net.add(FCLayer(15, 15, sigmoid, sigmoid_prime))
net.add(FCLayer(15, predictions, identity, identity_prime))

# train
net.use(mse, mse_prime, 0.55)
net.fit(X_train_normalized, y_train_arr, epochs=500, learning_rate=0.012, print_erro=True)

# test
out = net.predict(X_train_normalized)
print(out)

epoch 1/5000   error=123.221308
epoch 2/5000   error=205.587434
epoch 3/5000   error=237.429153
epoch 4/5000   error=241.894930
epoch 5/5000   error=236.271789
epoch 6/5000   error=222.139652
epoch 7/5000   error=211.452704
epoch 8/5000   error=207.883474
epoch 9/5000   error=200.462776
epoch 10/5000   error=196.099800
epoch 11/5000   error=190.265747
epoch 12/5000   error=186.762036
epoch 13/5000   error=183.889405
epoch 14/5000   error=182.315642
epoch 15/5000   error=181.166291
epoch 16/5000   error=180.175070
epoch 17/5000   error=179.264251
epoch 18/5000   error=178.451896
epoch 19/5000   error=177.789774
epoch 20/5000   error=177.290170
epoch 21/5000   error=176.921564
epoch 22/5000   error=176.636491
epoch 23/5000   error=176.394115
epoch 24/5000   error=176.159032
epoch 25/5000   error=175.892531
epoch 26/5000   error=175.555907
epoch 27/5000   error=175.123643
epoch 28/5000   error=174.591314
epoch 29/5000   error=173.971571
epoch 30/5000   error=173.284319
epoch 31/5000   err